In [2]:
import torch

In [4]:
# tensor of (2,3) size
t1 = torch.zeros(size=(2,3), dtype = torch.float32)
t2 = torch.ones(size=(2,3), dtype = torch.float32)
t3 = torch.rand(size=(2,3), dtype = torch.float32)
print(t1,t2,t3)

tensor([[0., 0., 0.],
        [0., 0., 0.]]) tensor([[1., 1., 1.],
        [1., 1., 1.]]) tensor([[0.5720, 0.2605, 0.2653],
        [0.4669, 0.1451, 0.3060]])


In [7]:
# porp of a tensor
print(t2.size(), t2.device, t2.dtype)

torch.Size([2, 3]) cpu torch.float32


In [13]:
import numpy as np

In [9]:
# Transform a tensor to a GPU tensor
t2 = t2.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [16]:
# From NumPy to tensor and viceversa
np_arr = np.random.rand(2,3)
print(np_arr.shape)

(2, 3)


In [17]:
t = torch.from_numpy(np_arr)
t.size()

torch.Size([2, 3])

In [18]:
np_arr_again = t.numpy()
np_arr_again.shape

(2, 3)

## Neural networks and Backpropagation
- Define network architecture
- While Training
    1. Forward pass : feed input and obatain net prediction
    2. Compute loss : compare prediction with the "Ground Truth"(label)
    3. Backward padss : compute gradient of the Network parameters w.r.t to the loss function(Pytorch autograd)
    4. Update the network params(Pytorch Optimizer)
    5. Repeat

### Computational Graph
- Support for forward and backward
- is a directed graph keeping track of all operation performed on Variables
- Nodes represet: 
    1. Variables
    2. Operations
- Pytorch Autograd handled this, providing automatic differentation

In [19]:
from torch.autograd import Variable

In [23]:
# graph crated on the fly Wh h Wx x
x = Variable(torch.rand(1,10))
prev_h = Variable(torch.rand(1,20))
W_h = Variable(torch.rand(20,20))
W_x = Variable(torch.rand(20,10))

### Basic compontens
- Net architecture (torch.nn.Module)
- Dataset (torch.utils.data.Dataset)
- Loss function + optimizer
- Training lopp


## Network form scratch

In [24]:
import torch.nn as nn

In [91]:
class NeuralNet(nn.Module):
    # inizialization of network layer
    def __init__(self,input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    # function descibing Input data path through net
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out



In [92]:
input_size, hidden_size, num_classes = 784, 500, 10
model = NeuralNet(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)
print(model)

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [93]:
from PIL import Image
import torchvision.transforms.functional as TF

In [94]:
image = Image.open('img/cat.jpg')
cat = TF.to_tensor(image)
cat.unsqueeze_(0)
print(cat.shape)

torch.Size([1, 3, 400, 640])


In [95]:
cat = torch.rand(784)
cat = Variable(cat.view(-1, 28*28))
# cat should be 28*28

In [97]:
forward_res = model(cat)
forward_res

tensor([[-0.2277,  0.0003,  0.0258,  0.0262,  0.1978, -0.0144,  0.0371, -0.1290,
         -0.0320,  0.0545]], grad_fn=<AddmmBackward>)

In [98]:
# Dataset
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [102]:
class DatasetMNIST(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        # load image ( Height * Width * Channels)
        # (H, W, C) => (C, H, W)
        image = self.data.iloc[index,1:].values.astype(np.uint8).reshape(1, 28, 28)
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image,label
        

In [105]:
transform = transforms.Compose([transforms.ToPILImage(), 
                                transforms.RandomHorizontalFlip(0.5), 
                                transforms.ToTensor()])
train_dataset = DatasetMNIST(file_path='input/train.csv', transform=transform)

In [111]:
import torchvision
# dataloader efficent iterator
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', train=True,download=True,
                              transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize((0.1307,),(0.3081,))
                              ])),
batch_size=100, shuffle=True)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/home/pierg/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629434582/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [113]:
# Loss and Optimizer
loss = nn.CrossEntropyLoss()
input = torch.rand(3,5,requires_grad=True)
target = torch.empty(3,dtype=torch.long).random_(5)
output = loss(input, target) # compute loss
output.backward() # compute gradient backward

In [118]:
import torch.optim as optim
# optimizer update net parameters depending on the gradient of the backward step
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
var1 = torch.rand(5)
var2 = torch.rand(5)
optimizer = optim.Adam([var1,var2],lr=0.0001)

In [128]:
# Training LOOP 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(100):
    tot_loss, tot_samples = 0.0, 0
    for i, data in enumerate(train_loader):
        # Step 1 Batch of input from dataloader
        images, labels = data
        tot_samples += inputs.size(0)
        
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        # Step 2 zero grading parametes always do before backward()
        optimizer.zero_grad()
        
        # Step 3 get prediction
        outputs = model(images)
        
        # Step 4 compute loss
        loss = criterion(outputs, labels)
        
        # Step 5 compute gradients
        loss.backward()
        
        # Step 6 optimize
        optimizer.step()
        
        #logging
        tot_loss += (loss.item()*inputs.size(0))
    # End epoch
    print("Epoch %d loss is: %.6f"%(epoch,(tot_loss*1.0/float(tot_samples))))
#End Training
        

KeyboardInterrupt: 

In [127]:
torch.save(model.state_dict(),'./model/fnn_model.pkl')